In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry

%matplotlib widget


In [2]:
def forced_photometry(diff, x, y):

    fluxes = []
    
    for i in range(len(diff)):
        if np.isnan(diff[i]).sum() > diff[i].shape[0] * diff[i].shape[1] * 0.5:
            fluxes.append(np.nan)
            continue

        aperture = CircularAperture([x, y], 1.91)
        
        phot_table = aperture_photometry(diff[i], aperture)
        phot_table = phot_table.to_pandas()
        flux = phot_table['aperture_sum'].values[0]
        fluxes.append(flux)
        
    return np.array(fluxes)

In [3]:
def check_lc_significance(start, end, x, y, diff, flux_sign = 1):#, buffer = 1, base_range=1.5, grad_val = -300):
    
    buffer = 53
    base_range = 80
    
    lc = forced_photometry(diff, x, y)
    
    lc[np.isnan(lc)] = 0
    
    gradients = np.gradient(lc)
    
    values = gradients < -100
    
    # Setting up the light curve
    frame_start = start - buffer
    frame_end = end + buffer
    if frame_start < 0:
        frame_start = 0
        frame_end += buffer
    if frame_end > len(lc):
        frame_end = len(lc) - 1 
        frame_start -= buffer
    
    if (frame_start < 0):
        frame_start = 0
    if (frame_end > len(lc)):
        frame_end = len(lc) - 1 
    
    baseline_start = frame_start - base_range
    baseline_end = frame_end + base_range
    if baseline_start < 0:
        baseline_start = 0
    if baseline_end > len(lc):
        baseline_end = len(lc) - 1 
    
    frames = np.arange(len(lc))
    ind = ((frames > baseline_start) & (frames < frame_start)) | ((frames < baseline_end) & (frames > frame_end))
    med = np.nanmedian(lc[ind])
    std = np.nanstd(lc[ind], ddof = 1)
    lcevent = lc[int(start):int(end)]
    
    # Light curve significance
    lc_sig = (lcevent - med) / std

    if flux_sign >= 0:
        sig_max = np.nanmax(lc_sig)
        sig_med = np.nanmean(lc_sig)
        
    else:
        sig_max = abs(np.nanmin(lc_sig))
        sig_med = abs(np.nanmean(lc_sig))
    
    lc_sig = (lc - med) / std
    return sig_max, sig_med, lc_sig * flux_sign, values

In [4]:
filtering_files = '/Users/zgl12/Modules/Kakapo/csv_files/c3/c3_t205922648.csv'
diff_file = '/Users/zgl12/Modules/Kakapo/difference_arrays/c3/diff_c3_t205922648.npy'

df = pd.read_csv(filtering_files)
diff = np.load(diff_file)

In [5]:
filtered_df = df[(df['frame'] > 2000) & (abs(df['xcentroid'] - 6.8) < 0.8) & (abs(df['ycentroid'] - 6.8) < 0.8)]

x = filtered_df['xcentroid'].mean()
y = filtered_df['ycentroid'].mean()

x_std = filtered_df['xcentroid'].std()
y_std = filtered_df['ycentroid'].std()

start = filtered_df['frame'].min()
end = filtered_df['frame'].max()

In [ ]:
x, y, x_std, y_std

In [ ]:
filtered_df[(filtered_df['fwhm'] > 0.8) & 
            (filtered_df['roundness'] < 0.7) & 
            (filtered_df['snr'] > 4) & 
            (filtered_df['correlation'] > 0) & 
            (filtered_df['poisson_thresh'] > 2) &
            (filtered_df['psfdiff'] < 1)]

In [8]:
flux = forced_photometry(diff, x, y)
flux[np.isnan(flux)] = 0

gradients = np.gradient(flux)

lines = np.arange(len(flux))[gradients < -100]

In [12]:
a, b, c, d = check_lc_significance(start, end, x, y, diff, flux_sign = 1)

In [14]:
ind = np.where(c > 2)[0]

In [ ]:
ind

In [ ]:
plt.figure()
plt.plot(flux)
for line in lines:
    plt.axvline(line, color = 'r')
for i in ind:
    plt.axvline(i, color = 'C1', alpha = 0.1)
plt.show()